Concerting the entity tags of the train, test, and dev data to using BIO-tags.

In [17]:
import pandas as pd
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [18]:
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [3]:
train = pd.read_parquet("data/train.parquet")
dev = pd.read_parquet("data/dev.parquet")
test = pd.read_parquet("data/test.parquet")

In [14]:
train.head()

,text,ents,sents,tokens,spans,dagw_source,dagw_domain,dagw_source_full
0,Danmark skal bygges af maskinernes forsigtige ...,"[{'start': 0, 'end': 7, 'label': 'GPE'}]","[{'start': 0, 'end': 60}]","[{'id': 0, 'start': 0, 'end': 7}, {'id': 1, 's...",{'incorrect_spans': []},danavis,News,Danish daily newspapers
1,Hvil i Cap Dag - 2011 år Cap D'Agde Privat bil...,"[{'start': 17, 'end': 21, 'label': 'DATE'}, {'...","[{'start': 0, 'end': 79}]","[{'id': 0, 'start': 0, 'end': 4}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl
2,Måtte det nye år gøre dem mere fortrolige med ...,"[{'start': 6, 'end': 16, 'label': 'DATE'}, {'s...","[{'start': 0, 'end': 171}]","[{'id': 0, 'start': 0, 'end': 5}, {'id': 1, 's...",{'incorrect_spans': []},naat,Conversation,NAAT
3,Vi har også 360 graders Private Banking rådgiv...,"[{'start': 12, 'end': 23, 'label': 'QUANTITY'}]","[{'start': 0, 'end': 117}]","[{'id': 0, 'start': 0, 'end': 2}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl
4,| Nyhedsmails,[],"[{'start': 0, 'end': 13}]","[{'id': 0, 'start': 0, 'end': 1}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl


In [25]:
for row in train.iloc:
    text = row["text"]
    ents = row["ents"]

    token_start = 0

    tokenized_text = bert_tokenizer(text).tokens()
    no_special_tokens = tokenized_text[1:-1]
    pure_tokens = []

    for i, token in enumerate(no_special_tokens):
        first_letter = token[0]

        if first_letter == '#':
            pure_token = token.strip('#')
            combined_token = pure_tokens[-1] + pure_token
            pure_tokens.pop()
            pure_tokens.append(combined_token)

        else:
            pure_tokens.append(token)

    
    new_ents = []
    for ent in ents:
        if token in ("[CLS]", "[SEP]"):
            # skip to next token
            continue 
        if token.startswith("##"):
            token = token[2:]

['[CLS]',
 'Vi',
 'har',
 'også',
 '360',
 'grader',
 '##s',
 'Private',
 'Bank',
 '##ing',
 'råd',
 '##gi',
 '##vning',
 'til',
 'form',
 '##uen',
 '##de',
 'kunde',
 '##r',
 'med',
 'specialist',
 '##er',
 'inden',
 '##for',
 'ska',
 '##t',
 'og',
 'in',
 '##vest',
 '##ering',
 '[SEP]']

In [41]:
tokenized_text = bert_tokenizer(train['text'][3]).tokens()
no_special_tokens = tokenized_text[1:-1]

pure_tokens = []
for i, token in enumerate(no_special_tokens):
    first_letter = token[0]
    if first_letter == '#':
        pure_token = token.strip('#')
        combined_token = pure_tokens[-1] + pure_token
        pure_tokens.pop()
        pure_tokens.append(combined_token)
    else:
        pure_tokens.append(token)

['[CLS]', 'Vi', 'har', 'også', '360', 'grader', '##s', 'Private', 'Bank', '##ing', 'råd', '##gi', '##vning', 'til', 'form', '##uen', '##de', 'kunde', '##r', 'med', 'specialist', '##er', 'inden', '##for', 'ska', '##t', 'og', 'in', '##vest', '##ering', '[SEP]']
['Vi', 'har', 'også', '360', 'grader', '##s', 'Private', 'Bank', '##ing', 'råd', '##gi', '##vning', 'til', 'form', '##uen', '##de', 'kunde', '##r', 'med', 'specialist', '##er', 'inden', '##for', 'ska', '##t', 'og', 'in', '##vest', '##ering']
['Vi', 'har', 'også', '360', 'graders', 'Private', 'Banking', 'rådgivning', 'til', 'formuende', 'kunder', 'med', 'specialister', 'indenfor', 'skat', 'og', 'investering']


In [45]:
ents = train['ents'][3]
print(ents)
ent = ents[0]
print(ent)
new_ents = []

count = 0
start = ent['start']
end = ent['end']
tag = ent['label']

for token in pure_tokens:
    new_ent_dict = {}
    token_len = len(token)
    new_count = count + token_len

    if count == start and new_count <= end:
        new_ent_dict['start'] = count
        new_ent_dict['end'] = count + new_count
        new_ent_dict['label'] = 'B-' + tag

    elif count > start and new_count < end:
        new_ent_dict['start'] = count
        new_ent_dict['end'] = count + new_count
        new_ent_dict['label'] = 'I-' + tag
    
    count += tok

SyntaxError: invalid syntax (2055908776.py, line 8)

In [40]:
for i in range(10):
    for j in range(3):
        if

['ko', 'hest']